
# Bayesian Networks Part Two

(The main notbook are reused from pgmpy package.)


This section will be about obtaining a Bayesian network, given a set of sample data. Learning a Bayesian network can be split into two problems:

 **Parameter learning:** Given a set of data samples and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.
 
 **Structure learning:** Given a set of data samples, estimate a DAG that captures the dependencies between the variables.
 
This notebook aims to illustrate how parameter learning and structure learning can be done with pgmpy.
Currently, the library supports:
 - Parameter learning for *discrete* nodes:
   - Maximum Likelihood Estimation
   - Bayesian Estimation
 - Structure learning for *discrete*, *fully observed* networks:
   - Score-based structure estimation (BIC/BDeu/K2 score; exhaustive search, hill climb/tabu search)
   - Constraint-based structure estimation (PC)


## Parameter Learning

Suppose we have the following data:

In [2]:
import pandas as pd
data = pd.DataFrame(data={'fruit': ["banana", "apple", "banana", "apple", "banana","apple", "banana", 
                                    "apple", "apple", "apple", "banana", "banana", "apple", "banana",], 
                          'tasty': ["yes", "no", "yes", "yes", "yes", "yes", "yes", 
                                    "yes", "yes", "yes", "yes", "no", "no", "no"], 
                          'size': ["large", "large", "large", "small", "large", "large", "large",
                                    "small", "large", "large", "large", "large", "small", "small"]})
print(data)

     fruit   size tasty
0   banana  large   yes
1    apple  large    no
2   banana  large   yes
3    apple  small   yes
4   banana  large   yes
5    apple  large   yes
6   banana  large   yes
7    apple  small   yes
8    apple  large   yes
9    apple  large   yes
10  banana  large   yes
11  banana  large    no
12   apple  small    no
13  banana  small    no


We know that the variables relate as follows:

In [3]:
from pgmpy.models import BayesianModel

model = BayesianModel([('fruit', 'tasty'), ('size', 'tasty')])  # fruit -> tasty <- size

Parameter learning is the task to estimate the values of the conditional probability distributions (CPDs), for the variables `fruit`, `size`, and `tasty`. 

#### State counts
To make sense of the given data, we can start by counting how often each state of the variable occurs. If the variable is dependent on parents, the counts are done conditionally on the parents states, i.e. for seperately for each parent configuration:

In [4]:
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, data)
print("\n", pe.state_counts('fruit'))  # unconditional
print("\n", pe.state_counts('tasty'))  # conditional on fruit and size

('\n',         fruit
apple       7
banana      7)
('\n', fruit apple       banana      
size  large small  large small
tasty                         
no      1.0   1.0    1.0   1.0
yes     3.0   2.0    5.0   0.0)


We can see, for example, that as many apples as bananas were observed and that `5` large bananas were tasty, while only `1` was not.

#### Maximum Likelihood Estimation

A natural estimate for the CPDs is to simply use the *relative frequencies*, with which the variable states have occured. We observed `7 apples` among a total of `14 fruits`, so we might guess that about `50%` of `fruits` are `apples`.

This approach is *Maximum Likelihood Estimation (MLE)*. According to MLE, we should fill the CPDs in such a way, that $P(\text{data}|\text{model})$ is maximal. This is achieved when using the *relative frequencies*. See [1], section 17.1 for an introduction to ML parameter estimation. pgmpy supports MLE as follows:

In [5]:
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, data)
print(mle.estimate_cpd('fruit'))  # unconditional
print(mle.estimate_cpd('tasty'))  # conditional

+---------------+-----+
| fruit(apple)  | 0.5 |
+---------------+-----+
| fruit(banana) | 0.5 |
+---------------+-----+
+------------+--------------+----------------+----------------+---------------+
| fruit      | fruit(apple) | fruit(apple)   | fruit(banana)  | fruit(banana) |
+------------+--------------+----------------+----------------+---------------+
| size       | size(large)  | size(small)    | size(large)    | size(small)   |
+------------+--------------+----------------+----------------+---------------+
| tasty(no)  | 0.25         | 0.333333333333 | 0.166666666667 | 1.0           |
+------------+--------------+----------------+----------------+---------------+
| tasty(yes) | 0.75         | 0.666666666667 | 0.833333333333 | 0.0           |
+------------+--------------+----------------+----------------+---------------+


`mle.estimate_cpd(variable)` computes the state counts and divides each cell by the (conditional) sample size. The `mle.get_parameters()`-method returns a list of CPDs for all variable of the model.

The built-in `fit()`-method of `BayesianModel` provides more convenient access to parameter estimators:


In [6]:
# Calibrate all CPDs of `model` using MLE:
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, data)
print(mle.estimate_cpd('fruit'))  # unconditional
print(mle.estimate_cpd('tasty'))  # conditional

+---------------+-----+
| fruit(apple)  | 0.5 |
+---------------+-----+
| fruit(banana) | 0.5 |
+---------------+-----+
+------------+--------------+----------------+----------------+---------------+
| fruit      | fruit(apple) | fruit(apple)   | fruit(banana)  | fruit(banana) |
+------------+--------------+----------------+----------------+---------------+
| size       | size(large)  | size(small)    | size(large)    | size(small)   |
+------------+--------------+----------------+----------------+---------------+
| tasty(no)  | 0.25         | 0.333333333333 | 0.166666666667 | 1.0           |
+------------+--------------+----------------+----------------+---------------+
| tasty(yes) | 0.75         | 0.666666666667 | 0.833333333333 | 0.0           |
+------------+--------------+----------------+----------------+---------------+



While very straightforward, the ML estimator has the problem of *overfitting* to the data. In above CPD, the probability of a large banana being tasty is estimated at `0.833`, because `5` out of `6` observed large bananas were tasty. Fine. But note that the probability of a small banana being tasty is estimated at `0.0`, because we  observed only one small banana and it happened to be not tasty. But that should hardly make us certain that small bananas aren't tasty!
We simply do not have enough observations to rely on the observed frequencies. If the observed data is not representative for the underlying distribution, ML estimations will be extremly far off. 

When estimating parameters for Bayesian networks, lack of data is a frequent problem. Even if the total sample size is very large, the fact that state counts are done conditionally for each parents configuration causes immense fragmentation. If a variable has 3 parents that can each take 10 states, then state counts will be done seperately for `10^3 = 1000` parents configurations. This makes MLE very fragile and unstable for learning Bayesian Network parameters. A way to mitigate MLE's overfitting is *Bayesian Parameter Estimation*.

#### Bayesian Parameter Estimation

The Bayesian Parameter Estimator starts with already existing prior CPDs, that express our beliefs about the variables *before* the data was observed. Those "priors" are then updated, using the state counts from the observed data. See [1], Section 17.3 for a general introduction to Bayesian estimators.

One can think of the priors as consisting in *pseudo state counts*, that are added to the actual counts before normalization.
Unless one wants to encode specific beliefs about the distributions of the variables, one commonly chooses uniform priors, i.e. ones that deem all states equiprobable.

A very simple prior is the so-called *K2* prior, which simply adds `1` to the count of every single state.
A somewhat more sensible choice of prior is *BDeu* (Bayesian Dirichlet equivalent uniform prior). For BDeu we need to specify an *equivalent sample size* `N` and then the pseudo-counts are the equivalent of having observed `N` uniform samples of each variable (and each parent configuration). In pgmpy:




In [7]:
from pgmpy.estimators import BayesianEstimator
est = BayesianEstimator(model, data)

print(est.estimate_cpd('tasty', prior_type='BDeu', equivalent_sample_size=10))

+------------+----------------+----------------+----------------+----------------+
| fruit      | fruit(apple)   | fruit(apple)   | fruit(banana)  | fruit(banana)  |
+------------+----------------+----------------+----------------+----------------+
| size       | size(large)    | size(small)    | size(large)    | size(small)    |
+------------+----------------+----------------+----------------+----------------+
| tasty(no)  | 0.346153846154 | 0.409090909091 | 0.264705882353 | 0.642857142857 |
+------------+----------------+----------------+----------------+----------------+
| tasty(yes) | 0.653846153846 | 0.590909090909 | 0.735294117647 | 0.357142857143 |
+------------+----------------+----------------+----------------+----------------+


The estimated values in the CPDs are now more conservative. In particular, the estimate for a small banana being not tasty is now around `0.64` rather than `1.0`. Setting `equivalent_sample_size` to `10` means that for each parent configuration, we add the equivalent of 10 uniform samples (here: `+5` small bananas that are tasty and `+5` that aren't).

`BayesianEstimator`, too, can be used via the `fit()`-method. Full example:

##### Parameters
----------
prior_type: 'dirichlet', 'BDeu', or 'K2'

string indicting which type of prior to use for the model parameters.
            
- If 'prior_type' is 'dirichlet', the following must be provided:

'pseudo_counts' = dirichlet hyperparameters; a dict containing, for each variable, a list with a "virtual" count for each variable state, that is added to the state counts.(lexicographic ordering of states assumed)
            
- If 'prior_type' is 'BDeu', then an 'equivalent_sample_size' must be specified instead of 'pseudo_counts'. This is equivalent to 'prior_type=dirichlet' and using uniform 'pseudo_counts' of `equivalent_sample_size/(node_cardinality*np.pro(parents_cardinalities))` for each node.
'equivalent_sample_size' can either be a numerical value or a dict that specifies the size for each variable seperately.

- A prior_type of 'K2' is a shorthand for 'dirichlet' + setting every pseudo_count to 1,regardless of the cardinality of the variable.

##### Returns
-------
parameters: list

List of TabularCPDs, one for each variable of the model


In [8]:
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator

# generate data
data = pd.DataFrame(np.random.randint(low=0, high=2, size=(5000, 4)), columns=['A', 'B', 'C', 'D'])
model = BayesianModel([('A', 'B'), ('A', 'C'), ('D', 'C'), ('B', 'D')])

model.fit(data, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=100)
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)


CPD of A:
+------+----------+
| A(0) | 0.497451 |
+------+----------+
| A(1) | 0.502549 |
+------+----------+
CPD of C:
+------+--------------+----------------+----------------+----------------+
| A    | A(0)         | A(0)           | A(1)           | A(1)           |
+------+--------------+----------------+----------------+----------------+
| D    | D(0)         | D(1)           | D(0)           | D(1)           |
+------+--------------+----------------+----------------+----------------+
| C(0) | 0.4969348659 | 0.512581168831 | 0.486263736264 | 0.489138867339 |
+------+--------------+----------------+----------------+----------------+
| C(1) | 0.5030651341 | 0.487418831169 | 0.513736263736 | 0.510861132661 |
+------+--------------+----------------+----------------+----------------+
CPD of B:
+------+----------------+----------------+
| A    | A(0)           | A(1)           |
+------+----------------+----------------+
| B(0) | 0.506897910918 | 0.514241123683 |
+------+---------------

In [9]:
pseudo_counts = {'A': [3000, 7000], 'B': [700, 300], 'C': [800, 200], 'D': [500, 500]}
model.fit(data, estimator=BayesianEstimator, prior_type='dirichlet', pseudo_counts=pseudo_counts)
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of A:
+------+--------+
| A(0) | 0.3658 |
+------+--------+
| A(1) | 0.6342 |
+------+--------+
CPD of C:
+------+----------------+---------------+----------------+----------------+
| A    | A(0)           | A(0)          | A(1)           | A(1)           |
+------+----------------+---------------+----------------+----------------+
| D    | D(0)           | D(1)          | D(0)           | D(1)           |
+------+----------------+---------------+----------------+----------------+
| C(0) | 0.629824561404 | 0.64295423652 | 0.625611382837 | 0.626325088339 |
+------+----------------+---------------+----------------+----------------+
| C(1) | 0.370175438596 | 0.35704576348 | 0.374388617163 | 0.373674911661 |
+------+----------------+---------------+----------------+----------------+
CPD of B:
+------+----------------+---------------+
| A    | A(0)           | A(1)          |
+------+----------------+---------------+
| B(0) | 0.562374533983 | 0.56732137774 |
+------+----------------+---

### Practice One:

In [10]:
np.random.seed(100)
raw_data = np.random.randint(low=0, high=2, size=(1000, 3))
data = pd.DataFrame(raw_data, columns=['D', 'I', 'G'])
data.D=data.I+data.G
np.random.seed(1000)
raw_data = np.random.randint(low=0, high=2, size=(20, 3))
data_test = pd.DataFrame(raw_data, columns=['D', 'I', 'G'])
data_test.I=1
data_test.D=data_test.I+data_test.G
print(data.head())
model = BayesianModel([("I","D"),("G","D")])

   D  I  G
0  1  0  1
1  2  1  1
2  0  0  0
3  1  0  1
4  0  0  0


In [11]:
?model.predict

### Q1. Please fit the data using BayesianEstimator. What are your CPDs. (use BDeu: 100 for example)? 

### Q2. 

a) Predict "G" using "I" by your model. What is the OS accuracy of prediction? What if use "I" and "D"?

b) Predict "D" using "G" by your model. What is the OS accuracy of prediction? What if use "I" and "G"?

c) What if you have the following sampling observations as prior?

pseudo_counts = {'D': [2000,1000,2000], 'I': [300, 700], 'G': [100, 100]}


In [12]:
model.fit(data, estimator=BayesianEstimator, prior_type='BDeu', equivalent_sample_size=10)
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of I:
+------+---------+
| I(0) | 0.50099 |
+------+---------+
| I(1) | 0.49901 |
+------+---------+
CPD of D:
+------+------------------+------------------+------------------+------------------+
| G    | G(0)             | G(0)             | G(1)             | G(1)             |
+------+------------------+------------------+------------------+------------------+
| I    | I(0)             | I(1)             | I(0)             | I(1)             |
+------+------------------+------------------+------------------+------------------+
| D(0) | 0.993698802773   | 0.00309214594929 | 0.00345065562457 | 0.00355366027008 |
+------+------------------+------------------+------------------+------------------+
| D(1) | 0.00315059861374 | 0.993815708101   | 0.993098688751   | 0.00355366027008 |
+------+------------------+------------------+------------------+------------------+
| D(2) | 0.00315059861374 | 0.00309214594929 | 0.00345065562457 | 0.99289267946    |
+------+------------------+--------

In [13]:
print (model.predict(data_test.loc[:,["I"]]).G==data_test.G).sum()*1.0/20
print (model.predict(data_test.loc[:,["D","I"]]).G==data_test.G).sum()*1.0/20

print (model.predict(data_test.loc[:,["G"]]).D==data_test.D).sum()*1.0/20
print (model.predict(data_test.loc[:,["I","G"]]).D==data_test.D).sum()*1.0/20

0.7
1.0
0.0
1.0


In [14]:
model = BayesianModel([("I","D"),("G","D")])
pseudo_counts = {'D': [2000,1000,2000], 'I': [300, 700], 'G': [100, 100]}
model.fit(data, estimator=BayesianEstimator, prior_type='dirichlet', pseudo_counts=pseudo_counts)
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

print (model.predict(data_test.loc[:,["I","G"]]).D==data_test.D).sum()*1.0/20


CPD of I:
+------+--------+
| I(0) | 0.4005 |
+------+--------+
| I(1) | 0.5995 |
+------+--------+
CPD of D:
+------+----------------+----------------+----------------+----------------+
| G    | G(0)           | G(0)           | G(1)           | G(1)           |
+------+----------------+----------------+----------------+----------------+
| I    | I(0)           | I(1)           | I(0)           | I(1)           |
+------+----------------+----------------+----------------+----------------+
| D(0) | 0.429874572406 | 0.379722802354 | 0.381752242794 | 0.382262996942 |
+------+----------------+----------------+----------------+----------------+
| D(1) | 0.190041809198 | 0.240554395291 | 0.236495514411 | 0.191131498471 |
+------+----------------+----------------+----------------+----------------+
| D(2) | 0.380083618396 | 0.379722802354 | 0.381752242794 | 0.426605504587 |
+------+----------------+----------------+----------------+----------------+
CPD of G:
+------+----------+
| G(0) | 0.52

## Structure Learning

To learn model structure (a DAG) from a data set, there are two broad techniques:

 - score-based structure learning
 - constraint-based structure learning

We briefly discuss all approaches and give examples.

### Score-based Structure Learning


This approach construes model selection as an optimization task. It has two building blocks:

- A _scoring function_ $s_D\colon M \to \mathbb R$ that maps models to a numerical score, based on how well they fit to a given data set $D$.
- A _search strategy_ to traverse the search space of possible models $M$ and select a model with optimal score.


#### Scoring functions

Commonly used scores to measure the fit between model and data are _Bayesian Dirichlet scores_ such as *BDeu* or *K2* and the _Bayesian Information Criterion_ (BIC, also called MDL). See [1], Section 18.3 for a detailed introduction on scores. As before, BDeu is dependent on an equivalent sample size.

In [15]:
import pandas as pd
import numpy as np
from pgmpy.estimators import BdeuScore, K2Score, BicScore
from pgmpy.models import BayesianModel

# create random data sample with 3 variables, where Z is dependent on X, Y:
data = pd.DataFrame(np.random.randint(0, 4, size=(5000, 2)), columns=list('XY'))
data['Z'] = data['X'] + data['Y']

bdeu = BdeuScore(data, equivalent_sample_size=5)
k2 = K2Score(data)
bic = BicScore(data)

model1 = BayesianModel([('X', 'Z'), ('Y', 'Z')])  # X -> Z <- Y
model2 = BayesianModel([('X', 'Z'), ('X', 'Y')])  # Y <- X -> Z


print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))

print(bdeu.score(model2))
print(k2.score(model2))
print(bic.score(model2))


-13934.6238246
-14325.3581389
-14290.6641388
-20895.3771637
-20922.22821
-20939.4281753


While the scores vary slightly, we can see that the correct `model1` has a much higher score than `model2`.
Importantly, these scores _decompose_, i.e. they can be computed locally for each of the variables given their potential parents, independent of other parts of the network:

In [16]:
print(bdeu.local_score('Z', parents=[]))
print(bdeu.local_score('Z', parents=['X']))
print(bdeu.local_score('Z', parents=['X', 'Y']))

-9246.48803524
-6988.45541056
-57.1166468066


#### Search strategies
The search space of DAGs is super-exponential in the number of variables and the above scoring functions allow for local maxima. The first property makes exhaustive search intractable for all but very small networks, the second prohibits efficient local optimization algorithms to always find the optimal structure. Thus, identifiying the ideal structure is often not tractable. Despite these bad news, heuristic search strategies often yields good results.

If only few nodes are involved (read: less than 5), `ExhaustiveSearch` can be used to compute the score for every DAG and returns the best-scoring one:

In [17]:
from pgmpy.estimators import ExhaustiveSearch

es = ExhaustiveSearch(data, scoring_method=bic)
best_model = es.estimate()
print(best_model.edges())

print("\nAll DAGs by score:")
for score, dag in reversed(es.all_scores()):
    print(score, dag.edges())

[('Y', 'Z'), ('X', 'Z')]

All DAGs by score:
(-14290.664138819098, [('Y', 'Z'), ('X', 'Z')])
(-14324.183352328386, [('Y', 'X'), ('Z', 'Y'), ('Z', 'X')])
(-14324.183352328388, [('Y', 'X'), ('Y', 'Z'), ('Z', 'X')])
(-14324.183352328388, [('Y', 'X'), ('Y', 'Z'), ('X', 'Z')])
(-14324.183352328388, [('X', 'Y'), ('Z', 'Y'), ('Z', 'X')])
(-14324.183352328389, [('X', 'Y'), ('X', 'Z'), ('Z', 'Y')])
(-14324.183352328389, [('Y', 'Z'), ('X', 'Y'), ('X', 'Z')])
(-16547.554933759962, [('Y', 'X'), ('Z', 'X')])
(-16550.498352353869, [('X', 'Y'), ('Z', 'Y')])
(-18682.537380375936, [('Z', 'Y'), ('Z', 'X')])
(-18682.537380375936, [('Y', 'Z'), ('Z', 'X')])
(-18682.537380375936, [('X', 'Z'), ('Z', 'Y')])
(-20905.908961807512, [('Z', 'X')])
(-20905.908961807512, [('X', 'Z')])
(-20908.852380401415, [('Z', 'Y')])
(-20908.852380401415, [('Y', 'Z')])
(-20939.428175316803, [('Y', 'X'), ('X', 'Z')])
(-20939.428175316803, [('X', 'Y'), ('Z', 'X')])
(-20939.428175316803, [('X', 'Y'), ('X', 'Z')])
(-20942.37159391070

Once more nodes are involved, one needs to switch to heuristic search. `HillClimbSearch` implements a greedy local search that starts from the DAG `start` (default: disconnected DAG) and proceeds by iteratively performing single-edge manipulations that maximally increase the score. The search terminates once a local maximum is found.




In [18]:
from pgmpy.estimators import HillClimbSearch

# create some data with dependencies
data = pd.DataFrame(np.random.randint(0, 3, size=(2500, 8)), columns=list('ABCDEFGH'))
data['A'] += data['B'] + data['C']
data['H'] = data['G'] - data['A']

hc = HillClimbSearch(data, scoring_method=BicScore(data))
best_model = hc.estimate()
print(best_model.edges())

[('A', 'C'), ('A', 'B'), ('C', 'B'), ('G', 'A'), ('H', 'A'), ('H', 'G')]


The search correctly identifies e.g. that `B` and `C` do not influnce `H` directly, only through `A` and of course that `D`, `E`, `F` are independent.


To enforce a wider exploration of the search space, the search can be enhanced with a tabu list. The list keeps track of the last `n` modfications; those are then not allowed to be reversed, regardless of the score. Additionally a `white_list` or `black_list` can be supplied to restrict the search to a particular subset or to exclude certain edges. The parameter `max_indegree` allows to restrict the maximum number of parents for each node.


### Constraint-based Structure Learning

A different, but quite straightforward approach to build a DAG from data is this:

1. Identify independencies in the data set using hypothesis tests 
2. Construct DAG (pattern) according to identified independencies

#### (Conditional) Independence Tests

Independencies in the data can be identified using chi2 conditional independence tests. To this end, constraint-based estimators in pgmpy have a `test_conditional_independence(X, Y, Zs)`-method, that performs a hypothesis test on the data sample. It allows to check if `X` is independent from `Y` given a set of variables `Zs`:

In [19]:
from pgmpy.estimators import ConstraintBasedEstimator

data = pd.DataFrame(np.random.randint(0, 3, size=(2500, 8)), columns=list('ABCDEFGH'))
data['A'] += data['B'] + data['C']
data['H'] = data['G'] - data['A']
data['E'] *= data['F']

est = ConstraintBasedEstimator(data)

print(est.test_conditional_independence('B', 'C'))          # independent
print(est.test_conditional_independence('B', 'H'))          # dependent
print(est.test_conditional_independence('B', 'E'))          # independent
print(est.test_conditional_independence('B', 'H', ['A']))   # independent
print(est.test_conditional_independence('A', 'G'))          # independent
print(est.test_conditional_independence('A', 'G',  ['H']))  # dependent

(1.7458642800596729, 0.98783644838898632, True)
(669.65055482303819, 1.6615465800288878e-124, True)
(2.0506421131182382, 0.99830988455518577, True)
(9.038666747654533, 0.99999999689457031, True)
(10.54523288135092, 0.95717478959691427, True)
(4628.0, 0.0, True)


`test_conditional_independence()` returns a tripel `(chi2, p_value, sufficient_data)`, consisting in the computed chi2 test statistic, the `p_value` of the test, and a heuristig flag that indicates if the sample size was sufficient. The `p_value` is the probability of observing the computed chi2 statistic (or an even higher chi2 value), given the null hypothesis that X and Y are independent given Zs.

This can be used to make independence judgements, at a given level of significance:

In [20]:
def is_independent(X, Y, Zs=[], significance_level=0.05):
    return est.test_conditional_independence(X, Y, Zs)[1] >= significance_level
print(is_independent('B', 'C'))
print(is_independent('B', 'H'))
print(is_independent('B', 'E'))
print(is_independent('B', 'H', ['A']))
print(is_independent('A', 'G'))
print(is_independent('A', 'G', ['H']))

True
False
True
True
True
False


#### DAG (pattern) construction

#### PC Algorithm for causal direction

With a method for independence testing at hand, we can construct a DAG from the data set in three steps:
1. Construct an undirected skeleton - `estimate_skeleton()`
2. Orient compelled edges to obtain partially directed acyclid graph (PDAG; I-equivalence class of DAGs) - `skeleton_to_pdag()`


####  The following step orients any remaining edges, essentially at random, but in a way which is consistent with the edge directions produced by PC.  This is useful if we want to use the learned structure to perform inference, but we should not make any assumptions re. causality from the edges directed in the last step.

3. Extend DAG pattern to a DAG by conservatively orienting the remaining edges in some way - `pdag_to_dag()`

Step 1.&2. form the so-called PC algorithm, see [2], page 550. PDAGs are `DirectedGraph`s, that may contain both-way edges, to indicate that the orientation for the edge is not determined.




In [21]:
skel, seperating_sets = est.estimate_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

model = est.pdag_to_dag(pdag)
print("DAG edges:        ", model.edges())

('Undirected edges: ', [('A', 'C'), ('A', 'B'), ('A', 'H'), ('E', 'F'), ('G', 'H')])
('PDAG edges:       ', [('A', 'H'), ('C', 'A'), ('B', 'A'), ('E', 'F'), ('G', 'H'), ('F', 'E')])
('DAG edges:        ', [('A', 'H'), ('C', 'A'), ('B', 'A'), ('G', 'H'), ('F', 'E')])


#### The `estimate()`-method provides a shorthand for the three steps above and directly returns a `BayesianModel`. run these three steps separately rather than using the shortcut (estimate() function)



In [22]:
print(est.estimate(significance_level=0.01).edges())

[('A', 'H'), ('C', 'A'), ('B', 'A'), ('G', 'H'), ('F', 'E')]


PC PDAG construction is only guaranteed to work under the assumption that the identified set of independencies is *faithful*, i.e. there exists a DAG that exactly corresponds to it. Spurious dependencies in the data set can cause the reported independencies to violate faithfulness. It can happen that the estimated PDAG does not have any faithful completions (i.e. edge orientations that do not introduce new v-structures). In that case a warning is issued.

### Practice Two.

In [23]:
import pandas as pd
# Generating radom data with each variable have 2 states and equal probabilities for each state
from sklearn.cross_validation import train_test_split
data = pd.read_csv("https://serv.cusp.nyu.edu/classes/ML_2016_Spring/ML_2017/NYC_taxi_sample.csv",index_col=0)
data.manhattan=data.manhattan-1
train,test=train_test_split(data,random_state=999,test_size=0.2)
train.head()

C:\Users\lingjing\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,manhattan,tip,dist,speed,pass
1549869,0,1,1,2,1
1524693,1,1,2,2,1
2017453,1,3,1,2,2
1670807,1,4,4,3,1
3389132,0,3,2,2,1


### (1) Detect the structure. 

Why don't try some both ExhaustiveSearch and HillClimbSearch with different scoring_methods

### (2) Train parameters using your learnt structure and get all independencies.

### (3)Report the OS classification accuracy for "Manhattan" 

use: model.predict

The code contains a bug. Change all the class from 0 not from 1. Your current values are from 1.

### (4) Use PC Algorithm to detect causal direction among the features provided.

In [24]:
hc = HillClimbSearch(train, scoring_method=BicScore(train))
best_model = hc.estimate()
print(best_model.edges())

[('dist', 'speed'), ('manhattan', 'tip'), ('manhattan', 'pass'), ('speed', 'manhattan'), ('pass', 'tip')]


In [25]:
# Defining the model
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator

model = BayesianModel(best_model.edges())

# Learing CPDs using Maximum Likelihood Estimators
model.fit(train, estimator=MaximumLikelihoodEstimator)
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)
print model.get_independencies()

CPD of dist:
+---------+-----------+
| dist(1) | 0.305184  |
+---------+-----------+
| dist(2) | 0.232231  |
+---------+-----------+
| dist(3) | 0.13019   |
+---------+-----------+
| dist(4) | 0.109078  |
+---------+-----------+
| dist(5) | 0.0832747 |
+---------+-----------+
| dist(6) | 0.140042  |
+---------+-----------+
CPD of manhattan:
+--------------+----------------+----------------+----------------+---------------+----------------+-----------------+
| speed        | speed(1)       | speed(2)       | speed(3)       | speed(4)      | speed(5)       | speed(6)        |
+--------------+----------------+----------------+----------------+---------------+----------------+-----------------+
| manhattan(0) | 0.301324503311 | 0.352281226627 | 0.575835475578 | 0.74870017331 | 0.812154696133 | 0.90099009901   |
+--------------+----------------+----------------+----------------+---------------+----------------+-----------------+
| manhattan(1) | 0.698675496689 | 0.647718773373 | 0.424164524

In [26]:
test.index=range(len(test))
predict_data = test.copy()
predict_data.drop('manhattan', axis=1, inplace=True)
y_pred = model.predict(predict_data-1)
(y_pred["manhattan"]==test.manhattan).sum()*1.0/len(test)

0.69418386491557227

### Causal Direction

In [29]:
est = ConstraintBasedEstimator(train)
skel, seperating_sets = est.estimate_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

('Undirected edges: ', [('tip', 'pass'), ('dist', 'speed'), ('dist', 'manhattan'), ('manhattan', 'speed')])
('PDAG edges:       ', [('dist', 'speed'), ('dist', 'manhattan'), ('tip', 'pass'), ('manhattan', 'speed'), ('manhattan', 'dist'), ('speed', 'dist'), ('speed', 'manhattan'), ('pass', 'tip')])



### References

[1] Koller & Friedman, Probabilistic Graphical Models - Principles and Techniques, 2009

[2] Neapolitan, [Learning Bayesian Networks](http://www.cs.technion.ac.il/~dang/books/Learning%20Bayesian%20Networks&#40;Neapolitan,%20Richard&#41;.pdf), 2003

[3] Tsamardinos et al., [The max-min hill-climbing BN structure learning algorithm](http://www.dsl-lab.org/supplements/mmhc_paper/paper_online.pdf), 2005

